<a href="https://colab.research.google.com/github/Zach-Seymour/FDSFE_ZSeymour/blob/main/Copy_of_CNNgalaxies_classify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pylab as pl
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import Callback


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

In [ ]:
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.callbacks import LambdaCallback, EarlyStopping


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd /content/drive/MyDrive/


/content/drive/MyDrive


In [ ]:
cd FDSFE

/content/drive/MyDrive/FDSFE


/content/drive/Shareddrives/DSPS2021


In [ ]:
ims = np.load("galaxies.npy")

In [ ]:
ims.shape

(1000, 424, 424, 3)

In [ ]:
training_labels2 = pd.read_csv("training_solutions_rev1_short.csv")
training_labels2

nga       spi       ell       irr
0      0.000000  0.512101  0.945547  0.027227
1      0.009222  0.236781  0.810851  0.140353
2      0.056931  0.117790  1.000000  0.000000
3      0.068059  0.125280  0.679602  0.128159
4      0.066161  0.000000  0.970617  0.029383
...         ...       ...       ...       ...
61573  0.000000  0.260804  0.754266  0.175454
61574  0.000000  0.053406  0.872356  0.062095
61575  0.019783  0.347328  0.160814  0.604214
61576  0.000000  0.545764  0.136000  0.500256
61577  0.093000  0.070000  0.667000  0.204795

[61578 rows x 4 columns]

In [ ]:
newclass = [np.where(training_labels2.iloc[i] == np.max(training_labels2.iloc[i]))[0][0]
            for i in range(1000)]

In [ ]:
len(newclass)

1000

# split a 1000 galaxies dataset into train-test with a proportion of 2-1. Note that 1000 object is a critically small dataset for DNN!

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(ims, training_labels2.iloc[:1000].values, test_size=0.33, random_state=42)

In [ ]:
X_train.shape

(670, 424, 424, 3)

In [ ]:
img_rows, img_cols = X_train.shape[1], X_train.shape[2]
#x_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
#x_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
input_shape = X_train.shape[1:]


In [ ]:
X_train.shape

(670, 424, 424, 3)

In [ ]:
X_train = X_train / 255.0
X_test = X_test / 255.0


In [ ]:
batch_size = 8
num_classes = 4
epochs = 50
input_shape = ims[0].shape
input_shape

(424, 424, 3)

In [ ]:
y_train.shape

(670, 4)

In [ ]:

# Set Optimizer
opt = Adam(learning_rate=0.001)

In [ ]:
def build_cnn(input_shape, num_classes):
  inputs = layers.Input(shape = input_shape)

  x = layers.Conv2D(32,kernel_size = (3,3), activation = 'relu')(inputs)
  x = layers.Conv2D(32,kernel_size = (3,3), activation = 'relu')(x)
  x = layers.MaxPooling2D(pool_size = (2,2))(x)
  x = layers.Dropout()(x)
  x = layers.Flatten()(x)
  x = layers.Dense(512,activation = 'relu')(x)
  x = layers.Dropout()(x)
  x = layers.Dense(512,activation = 'relu')(x)
  outputs = layers.Dense(num_classes, activation = 'softmax')(x)

  model = keras.Model(inputs, outputs)
  return model

model = build_cnn(input_shape, num_classes)

In [ ]:
model.build()
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 415, 415, 32)      9632      
                                                                 
 conv2d_3 (Conv2D)           (None, 413, 413, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 206, 206, 64)      0         
 g2D)                                                            
                                                                 
 dropout_2 (Dropout)         (None, 206, 206, 64)      0         
                                                                 
 flatten_1 (Flatten)         (None, 2715904)           0         
                                                                 
 dense_2 (Dense)             (None, 128)               347635840 
                                                      

In [ ]:
#callbacks
patience = EarlyStopping(patience=5) #stops running if metric does not improve for 5 epochs in a row
filepath = "galcheckpoints/cnn-improvement-{epoch:02d}-{loss:.4f}.hd5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1,
                             save_best_only=True, mode='min') # saves intermediate weights at each epoch (for the best batch output)

In [ ]:
# you can skip this and load the saved model
history = model.fit(...)#, checkpoint])

In [ ]:
#test_loss, test_acc = model.evaluate(X_test, y_test)

In [ ]:
pl.plot(history.history['accuracy'])

In [ ]:
score = model.evaluate(X_test[:10], y_test[:10])


In [ ]:
score

In [ ]:
from scipy import ndimage, misc
newimg = np.zeros([1000*5, img.shape[0], img.shape[1], img.shape[2]],
                  dtype=np.uint8)
np.random.seed(123)
for i in range(len(ims)):
  angles = np.random.rand(4) *   180
  newimg[i*5] = ims[i].copy()
  for j,a in enumerate(angles):

    newimg[i*5+1+j] = ndimage.rotate(newimg[i*5], a,
                                         reshape=False, mode='mirror')



In [ ]:
pl.imshow(ims[0])

In [ ]:

for i in range (10):
  pl.imshow(newimg[i])
  pl.show()

in a look like the one above, take each image and rotate if by a random angle, **but also change its contrast by a random amount**. see the example below. choose small contrast changes if you want your images to remain realistic

In [ ]:
fig = pl.figure()
# a power < 1 (here 0.2) lowers the contrast
imgnew = ims[10].astype(float)**0.6

#normalize: reset range to 0-255
imgnew *= 255 / imgnew.max()
imgnew = imgnew.astype(np.uint8)

pl.imshow(ims[10])

fig = pl.figure()
pl.imshow(imgnew)

